In [ ]:
# 1. Carregar melhor modelo
model_artifact = tuner.best_estimator().model_data

# 2. Deploy endpoint de teste
predictor = tuner.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',  # Mais barato para testes
    endpoint_name='cbis-test-endpoint'
)

# 3. Testar com imagens do test set
test_images = ["s3://bucket/test/image1.png", ...]
predictions = []
for img in test_images:
    pred = predictor.predict(img)
    predictions.append(pred)

# 4. Calcular métricas
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))